In [37]:
# Creem la conex amb la db MYSQL
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    username="root",
    password="123321",
    database="tenfe"
)

In [38]:
colors = {
    'R1': 'bg-cyan-600',
    'R3': 'bg-red-600',
    'R4': 'bg-orange-600',
    'R11': 'bg-sky-600',
    'R12': 'bg-yellow-300',
}

In [39]:
import pandas as pd
import numpy as np

In [40]:
# Llegim el excel i el guardem en un dataframe
df = pd.read_excel('./Horaris.xlsx')

In [41]:
# Creem un cursor per poder fer les consultes
cursor = db.cursor()

In [42]:
# Extraiem el ID_PARADA del dataframe i el guardem en una llista
parades = df['ID_PARADA'].unique()

parades

print(parades)

# Inserim les parades a la taula PARADES
for parada in parades:
    # Comprovem que no existeixi la parada
    print(parada)
    cursor.execute("SELECT * FROM stations WHERE name = %s", (parada,))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la parada
    sql = "INSERT INTO stations (name) VALUES (%s)"
    val = (parada,)
    cursor.execute(sql, val)


# Guardem els canvis
db.commit()

['L’Hospitalet de Llobregat' 'Barcelona Sants' 'Barcelona Pl.Catalunya'
 'Barcelona Arc de Triomf' 'Barcelona El Clot Aragó' 'St.Adrià de Besòs'
 'Badalona' 'Montgat' 'Montgat Nord' 'El Masnou' 'Premià de Mar'
 'Vilassar de Mar' 'Cabrera de Mar - Vilassar de Mar' 'Mataró'
 'St.Andreu de Llavaneres' 'Caldes d’Estrac' 'Arenys de Mar'
 'Canet de Mar' 'St.Pol de Mar' 'Calella' 'Pineda de mar' 'Santa Susanna'
 'Malgrat de Mar' 'Blanes' 'Tordera' 'Maçanet Massanes'
 'Barcelona La Sagrera-Meridiana' 'Barcelona St.Andreu Arenal'
 'Barcelona Torre del Baró' 'Montcada Bifurcació' 'Montcada Ripollet'
 'Sta.Perpètua de Mogoda' 'Mollet Sta.Rosa' 'Parets del Vallès'
 'Granollers - Canovelles' 'Les Franqueses del Vallès' 'La Garriga'
 'Figaró' 'St.Martí de Centelles' 'Centelles' 'Balenyà - Els Hostalets'
 'Balenyà - Tona - Seva' 'Vic' 'Manlleu' 'Torelló' 'St.Quirze de Besora'
 'Ripoll' 'Campdevànol' 'Ribes de Freser' 'Planoles' 'Toses' 'La Molina'
 'Urtx - Alp' 'Puigcerdà' 'La Tor de Querol' 'Borgony

In [43]:
# Canviem del df cada nom de parada pel seu id
for _, fila in df.iterrows():
    sql = "SELECT id FROM stations WHERE name = %s"
    val = (fila['ID_PARADA'],)
    cursor.execute(sql, val)
    result = cursor.fetchone()
    # Canviem de la fila el nom de la parada pel seu id
    try:
        df.loc[df['ID_PARADA'] == fila['ID_PARADA'], 'ID_PARADA'] = result[0]
    except:
        print("error en la fila: ", fila)

df

,LINIA,ID_PARADA,HORA,ID_VIAJE,ORDEN
0,R1,1,05:48:00,1,1
1,R1,2,05:55:00,1,2
2,R1,3,06:00:00,1,3
3,R1,4,06:03:00,1,4
4,R1,5,06:07:00,1,5
...,...,...,...,...,...
10503,R12,124,07:56:00,484,5
10504,R12,119,07:59:00,484,6
10505,R12,118,08:02:00,484,7
10506,R12,115,08:09:00,484,10


In [44]:
# A tota la columna HORA la passem a string i posteriorment li treiem els ultims 3 caracters
df['HORA'] = df['HORA'].astype(str)
df['HORA'] = df['HORA'].str[:-3]

In [45]:
# Creem la linia a la taula linies si no existeix
for _, fila in df.iterrows():
    # Comprovem que no existeixi la linia
    cursor.execute("SELECT * FROM routes WHERE name = %s", (fila['LINIA'],))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la linia
    sql = "INSERT INTO routes (name, colour) VALUES (%s, %s)"
    val = (fila['LINIA'], colors[fila['LINIA']])
    cursor.execute(sql, val)

In [ ]:
# Inserim les dades de l'excel a la db
for _, row in df.iterrows():
    # Abans que res comprovem que no existeixi el registre
    cursor.execute("SELECT * FROM schedules WHERE station_id = %s AND time = %s AND train_num = %s AND stop_number = %s", (row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN']))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim el registre
    sql = "INSERT INTO schedules (route_id, station_id, time, train_num, stop_number) VALUES (%s, %s, %s, %s, %s)"
    val = (row['LINIA'], row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN'])
    cursor.execute(sql, val)
    
db.commit()